In [2]:
import pandas as pd

In [3]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and p

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModelForMaskedLM
import numpy as np
from transformers import DataCollatorWithPadding
import evaluate

In [35]:
model_path = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label = {0:"OR", 1: "CG"} #OG - real review, CG - computer generated
label2id = {"OR":0, "CG":1}

model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#freeze all base parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

#unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
  if "pooler" in name:
    param.requires_grad = True

Data pre-processing

In [36]:
!gdown 1_qsilovsJTqmOGpW0bmEO4sTYLlTBFH0

Downloading...
From: https://drive.google.com/uc?id=1_qsilovsJTqmOGpW0bmEO4sTYLlTBFH0
To: /content/fake reviews dataset.csv
100% 15.3M/15.3M [00:00<00:00, 77.0MB/s]


In [37]:
df = pd.read_csv('/content/fake reviews dataset.csv')
df.columns

Index(['category', 'rating', 'label', 'text_'], dtype='object')

In [38]:
df.label.value_counts()

,count
label,
CG,20216
OR,20216


In [47]:
from datasets import Dataset

# Convert your Pandas DataFrame to a Hugging Face Dataset
df["label"] = df["label"].map(label2id)
dataset = Dataset.from_pandas(df)

def tokenize_function(examples):
    return tokenizer(examples["text_"], padding="max_length", truncation=True)

# Apply the tokenize_function in batches using Dataset.map()
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/40432 [00:00<?, ? examples/s]

In [48]:
print(tokenized_datasets[0])

{'category': 'Home_and_Kitchen_5', 'rating': 5.0, 'label': 1, 'text_': 'Love this!  Well made, sturdy, and very comfortable.  I love it!Very pretty', 'input_ids': [101, 2293, 2023, 999, 2092, 2081, 1010, 23073, 1010, 1998, 2200, 6625, 1012, 1045, 2293, 2009, 999, 2200, 3492, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [49]:
test_size = 0.2
train, test = tokenized_datasets.train_test_split(test_size=test_size).values()

In [50]:
#create data collator
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

Evaluation metrics defining

In [51]:
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
  #predictions
  predictions, labels =eval_pred

  #applying softmax to get probs
  probabilities = np.exp(predictions) / np.exp(predictions).sum(-1,keepdims=True)

  positive_class_probs = probabilities[:,1]

  #compute auc
  auc = np.round(auc_score.compute(prediction_scores=positive_class_probs,
                                   references = labels)['roc_auc'],3)
  #predict most probable class
  predicted_classes = np.argmax(predictions, axis=1)
  #compute accuracy
  acc = np.round(auc_score.compute(prediction_scores=positive_class_probs,
                                   references = labels)['accuracy'],3)

  return {"accuracy": acc, "auc": auc}

Training params

In [52]:
#hyperparams
lr = 2e-4
bath_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-fake-rr-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=bath_size,
    per_device_eval_batch_size=bath_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [53]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

<ipython-input-53-e234904ebeb9>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
